# Dependencies not already included with Colab

In [ ]:
!pip install ultralytics
!pip install sympy==1.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.9.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=96bbd5ec6b736031505213b123e1045b27c320837d388d9b7f669d275d82e8e7
  Stored in directory: /root/.cache/pip/wheels/b6/a1/81/1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


# Data Prep

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# streaming files from drive on colab was undoing the benefits
# full coco dataset far too large for time must use a subset
# subset created within my files and uploaded to drive now copy .zip

!mkdir -p /content/data/coco

!echo "Copying COCO subset zip..."
!rsync -ah --info=progress2 "/content/drive/MyDrive/data/cocoSubset.zip" "/content/data/coco/"


!echo "Copying KITTI dataset..."
!rsync -ah --info=progress2 "/content/drive/MyDrive/data/kitti.zip" "/content/data/"


Copying COCO subset zip...
          2.45G 100%   51.94MB/s    0:00:45 (xfr#1, to-chk=0/1)
Copying KITTI dataset...
         12.35G 100%   52.71MB/s    0:03:43 (xfr#1, to-chk=0/1)


In [ ]:
!unzip "/content/data/coco/cocoSubset.zip" -d "/content/data/coco"
!unzip "/content/data/kitti.zip" -d "/content/data/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/006437.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/002651.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/002889.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/004220.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/002645.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/004234.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/000052.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/006423.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/006345.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/000734.png  
  inflating: /content/data/kitti/data_object_image_2/testing/image_2/004552.png  
  inflating: /content/data/kitti/data_object_im

In [ ]:
# copying coco validation set
!mkdir /content/data/coco/val2017
!rsync -ah --info=progress2 "/content/drive/MyDrive/data/coco/val2017/" "/content/data/coco/val2017/"
!rsync -ah --info=progress2 "/content/drive/MyDrive/data/coco/annotations/instances_val2017.json" "/content/data/coco/annotations/

        814.71M 100%    4.15MB/s    0:03:07 (xfr#5000, to-chk=0/5001)
/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


Issues with the file uploaded through command just uploaded through system coco/annotations/instances_val2017.json

Now that all annotations are in the correct format YOLO expects file structure to be a certain way such as:  
dataset/  
├── images/  
│   ├── train/  
│   │   ├── 000001.jpg  
│   │   ├── 000002.jpg  
│   │   └── ...  
│   ├── val/  
│       ├── 000101.jpg  
│       ├── 000102.jpg  
│       └── ...  
│  
└── labels/  
    ├── train/  
    │   ├── 000001.txt  
    │   ├── 000002.txt  
    │   └── ...  
    ├── val/  
        ├── 000101.txt  
        ├── 000102.txt  
        └── ...  
Setting these up using the file explorer within colab

In [ ]:
import os
import yaml
import json
import shutil
import random
from PIL import Image

class dataPrep:
	# base for working in google colab will be different based on where the data is stored
	def __init__(self, base="/content/data"):
		self.base = base
		self.data = ["kitti", "coco"]

	def setup(self):
		# Ensure data exist
		os.makedirs(self.base, exist_ok=True)
		print(f"{self.base} created")

		# Confirm coco and kitti in data
		for d in self.data:
			os.makedirs(f"{self.base}/{d}", exist_ok=True)
			print(f"{d} verified")

	def cocoFormat(self, jPath, imgDir, labelDir, names):
		# ensure labels folder exists
		os.makedirs(labelDir, exist_ok=True)
		print(f"{labelDir} created")

		# load COCO JSON
		with open(jPath, "r") as f:
			d = json.load(f)

		# map COCO category IDs to YOLO class IDs
		idConvert = {}
		for c in d["categories"]:
			if c["name"] in names:
				idConvert[c["id"]] = names.index(c["name"])

		# map image IDs to file names
		idToFile = {i["id"]: i["file_name"] for i in d["images"]}
		idToInfo = {i["id"]: i for i in d["images"]}

		annCount = 0
		noAnnCount = 0

		# process each image
		for iId in idToFile:
			imgName = idToFile[iId]
			info = idToInfo[iId]
			iW, iH = info["width"], info["height"]

			# get annotations for this image with categories in names
			ann = [a for a in d["annotations"] if a["image_id"] == iId and a["category_id"] in idConvert]

			# path to output YOLO label file
			lPath = os.path.join(labelDir, imgName.replace(".jpg", ".txt"))

			# debug: print if no annotations
			if not ann:
				print(f"no annotations for {imgName}")
				noAnnCount += 1
				# still create an empty txt
				open(lPath, "w").close()
				continue
			else:
				annCount += 1

			# write YOLO labels
			with open(lPath, "w") as f:
				for a in ann:
					x, y, w, h = a["bbox"]
					xCenter = (x + w/2) / iW
					yCenter = (y + h/2) / iH
					width = w / iW
					height = h / iH
					id = idConvert[a["category_id"]]
					f.write(f"{id} {xCenter} {yCenter} {width} {height}\n")

		print("\n=== coco summary ===")
		print(f"successful annotations: {annCount}")
		print(f"missing annotations: {noAnnCount}")
		print(f"total images: {len(idToFile)}")

	def kittiFormat(self, sourceImg, sourceLabel, trainImg, trainLabel, valImg, valLabel, names, split=0.8):
		# ensure output directories exist
		os.makedirs(trainImg, exist_ok=True)
		os.makedirs(trainLabel, exist_ok=True)
		os.makedirs(valImg, exist_ok=True)
		os.makedirs(valLabel, exist_ok=True)
		print(f"directories created")

		# get all image files
		imgFiles = [f for f in os.listdir(sourceImg) if f.endswith(".png")]

		# shuffle for random split
		random.shuffle(imgFiles)

		# calculate split index
		splitIdx = int(len(imgFiles) * split)
		trainFiles = imgFiles[:splitIdx]
		valFiles = imgFiles[splitIdx:]

		print(f"\n=== kitti split ===")
		print(f"total images: {len(imgFiles)}")
		print(f"train images: {len(trainFiles)} ({len(trainFiles)/len(imgFiles)*100:.1f}%)")
		print(f"val images: {len(valFiles)} ({len(valFiles)/len(imgFiles)*100:.1f}%)")

		# process both train and val sets
		for setName, files, destImg, destLabel in [
			("training", trainFiles, trainImg, trainLabel),
			("validation", valFiles, valImg, valLabel)
		]:
			print(f"\nprocessing {setName} set...")
			for f in files:
				# source paths
				srcImgPath = os.path.join(sourceImg, f)
				srcLabelPath = os.path.join(sourceLabel, f.replace(".png", ".txt"))

				# destination paths
				destImgPath = os.path.join(destImg, f)
				destLabelPath = os.path.join(destLabel, f.replace(".png", ".txt"))

				# copy image
				shutil.copy(srcImgPath, destImgPath)

				# skip if no label file exists
				if not os.path.exists(srcLabelPath):
					print(f"no label found for {f}")
					open(destLabelPath, "w").close()
					continue

				# open image to get dimensions
				iFile = Image.open(srcImgPath)
				iW, iH = iFile.size

				# read kitti labels
				with open(srcLabelPath, "r") as fIn:
					lines = fIn.readlines()

				# write yolo labels
				with open(destLabelPath, "w") as fOut:
					for l in lines:
						l = l.strip().split()
						if len(l) < 8:
							continue

						name = l[0]

						# skip if class not used
						if name not in names:
							continue

						id = names.index(name)

						# kitti: xmin ymin xmax ymax
						xmin, ymin, xmax, ymax = map(float, l[4:8])

						# convert to yolo bbox
						w = xmax - xmin
						h = ymax - ymin
						xCenter = (xmin + w/2) / iW
						yCenter = (ymin + h/2) / iH
						w /= iW
						h /= iH

						# write yolo line
						fOut.write(f"{id} {xCenter} {yCenter} {w} {h}\n")

	def yaml(self, d, tPath, vPath, names):
		# yaml data
		yFile = {
			"path": self.base,
			"train": tPath,
			"val": vPath,
			"names": {i: name for i, name in enumerate(names)}
		}

		yPath = os.path.join(self.base, f"{d}.yaml")

		# write yaml
		with open(yPath, "w") as f:
			yaml.dump(yFile, f, sort_keys=False)

if __name__ == "__main__":
	dp = dataPrep()
	dp.setup()

	namesC = [
		"person","bicycle","car","motorcycle","airplane","bus","train","truck",
		"boat","traffic light","fire hydrant","stop sign","parking meter","bench",
		"bird","cat","dog","horse","sheep","cow","elephant","bear","zebra","giraffe",
		"backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard",
		"sports ball","kite","baseball bat","baseball glove","skateboard","surfboard",
		"tennis racket","bottle","wine glass","cup","fork","knife","spoon","bowl",
		"banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza",
		"donut","cake","chair","couch","potted plant","bed","dining table","toilet",
		"tv","laptop","mouse","remote","keyboard","cell phone","microwave","oven",
		"toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear",
		"hair drier","toothbrush"
	]

	namesK = ["Car", "Pedestrian", "Cyclist"]

	# Convert training coco
	dp.cocoFormat(
		jPath=f"{dp.base}/coco/annotations/instances_train2017_subset.json",
		imgDir=f"{dp.base}/coco/images/train2017Subset",
		labelDir=f"{dp.base}/coco/labels/train2017Subset",
		names=namesC
	)

	# convert validation coco
	dp.cocoFormat(
		jPath=f"{dp.base}/coco/annotations/instances_val2017.json",
		imgDir=f"{dp.base}/coco/images/val2017",
		labelDir=f"{dp.base}/coco/labels/val2017",
		names=namesC
	)

	# kitti split and convert
	dp.kittiFormat(
		sourceImg=f"{dp.base}/kitti/data_object_image_2/training/image_2",
		sourceLabel=f"{dp.base}/kitti/training/label_2",
		trainImg=f"{dp.base}/kitti/split/images/training",
		trainLabel=f"{dp.base}/kitti/split/labels/training",
		valImg=f"{dp.base}/kitti/split/images/testing",
		valLabel=f"{dp.base}/kitti/split/labels/testing",
		names=namesK,
		split=0.8
	)

	# create yaml
	dp.yaml(
		d="coco",
		tPath="coco/images/train2017Subset",
		vPath="coco/images/val2017",
		names=namesC
	)

	dp.yaml(
		d="kitti",
		tPath="kitti/split/images/training",
		vPath="kitti/split/images/testing",
		names=namesK
	)

/content/data created
kitti verified
coco verified
/content/data/coco/labels/train2017Subset created
no annotations for 000000222757.jpg
no annotations for 000000201632.jpg
no annotations for 000000115654.jpg
no annotations for 000000450098.jpg
no annotations for 000000016449.jpg
no annotations for 000000458540.jpg
no annotations for 000000362351.jpg
no annotations for 000000414754.jpg
no annotations for 000000242558.jpg
no annotations for 000000241595.jpg
no annotations for 000000097785.jpg
no annotations for 000000140922.jpg
no annotations for 000000247624.jpg
no annotations for 000000356834.jpg
no annotations for 000000253520.jpg
no annotations for 000000266611.jpg
no annotations for 000000381842.jpg
no annotations for 000000279263.jpg
no annotations for 000000254124.jpg
no annotations for 000000375096.jpg
no annotations for 000000431026.jpg
no annotations for 000000283147.jpg
no annotations for 000000049725.jpg
no annotations for 000000387416.jpg
no annotations for 000000570045.jpg

## YOLOv8 Model

In [ ]:
from ultralytics import YOLO

class yolo:
	def __init__(self, model="yolov8n.pt"):
		self.model = model  # store current model path / checkpoint

	def train(self, d, e, b, name):
		# load current model (base or checkpoint)
		model = YOLO(self.model)

		# train on the specified dataset
		results = model.train(
			data=f"{d}.yaml",
			epochs=e,
			imgsz=768,
			batch=b,
			name=name,
			lr0=0.002
		)

		# update model path to best weights from this run
		# ultralytics saves best weights at runs/detect/{name}/weights/best.pt
		self.model = f"runs/detect/{name}/weights/best.pt"

		print(f"Finished training: {name}")
		return results

	def trainer(self):
		self.train("/content/data/coco", 30, 24, "yoloCoco")
		self.train("/content/data/kitti", 60, 16, "yoloKitti")


if __name__ == "__main__":
	y = yolo()
	y.trainer()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/coco.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7,

zip final folders

In [ ]:
!zip -r /content/yoloCoco.zip /content/runs/detect/yoloCoco
!zip -r /content/yoloKitti.zip /content/runs/detect/yoloKitti

  adding: content/runs/detect/yoloCoco/ (stored 0%)
  adding: content/runs/detect/yoloCoco/train_batch12500.jpg (deflated 9%)
  adding: content/runs/detect/yoloCoco/train_batch2.jpg (deflated 2%)
  adding: content/runs/detect/yoloCoco/args.yaml (deflated 53%)
  adding: content/runs/detect/yoloCoco/results.png (deflated 6%)
  adding: content/runs/detect/yoloCoco/train_batch12502.jpg (deflated 8%)
  adding: content/runs/detect/yoloCoco/val_batch0_labels.jpg (deflated 13%)
  adding: content/runs/detect/yoloCoco/weights/ (stored 0%)
  adding: content/runs/detect/yoloCoco/weights/last.pt (deflated 9%)
  adding: content/runs/detect/yoloCoco/weights/best.pt (deflated 9%)
  adding: content/runs/detect/yoloCoco/labels.jpg (deflated 21%)
  adding: content/runs/detect/yoloCoco/val_batch2_labels.jpg (deflated 4%)
  adding: content/runs/detect/yoloCoco/val_batch0_pred.jpg (deflated 12%)
  adding: content/runs/detect/yoloCoco/confusion_matrix.png (deflated 20%)
  adding: content/runs/detect/yoloCoco